In [1]:
import pandas as pd
from google.cloud import bigquery

Cargamos un dataframe con el dataset de sonidos en New York

In [2]:
df = pd.read_csv('../Datasets/SONYCUrbanSoundTagging.csv')

Veamos información general del set de datos de los ruidos de NYC

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62022 entries, 0 to 62021
Data columns (total 78 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   split                                         62022 non-null  object 
 1   sensor_id                                     62022 non-null  int64  
 2   audio_filename                                62022 non-null  object 
 3   annotator_id                                  62022 non-null  int64  
 4   borough                                       62022 non-null  int64  
 5   block                                         62022 non-null  int64  
 6   latitude                                      62022 non-null  float64
 7   longitude                                     62022 non-null  float64
 8   year                                          62022 non-null  int64  
 9   week                                          62022 non-null 

La columna 'split' indica el conjunto al que pertenece la grabacion: entrenamiento, validación y
prueba.<br>Esta variable 'split' no aporta al analisis para el dashboard y en el modelo de prediccion.

Hay 56 dispositivos sensores que captan los sonidos de la ciudad.<br>Esta variable no aporta al analisis para el dashboard y en el modelo de prediccion.

Hay 18510 archivos de audio en formato .wav que contienen los sonidos grabados de NYC.<br>Esta variable 'audio_filename' no aporta al analisis para el dashboard y en el modelo de prediccion.

Hay 4158 personas que anotan. Los valores negativos indican que los sonidos fueron clasificados por miembros de SONYC. Si los valores son positivos lo sonidos fueron clasificados por colaboradores de Zooniverse.<br>Esta variable 'annotator_id' no aporta al analisis para el dashboard y en el modelo de prediccion.

Los sonidos distintos a motores pequeños y medianos no aportan al analisis para el dashboard y en el modelo de prediccion.

Solo nos quedaremos con las siguientes variables:

In [4]:
df_noise = df[[
    'borough', 'block', 'latitude', 'longitude', 'year', 'week', 'day', 'hour',
    '1-1_small-sounding-engine_presence', '1-2_medium-sounding-engine_presence']]

In [5]:
#Cambia el nombre de las columnas que empizan con un numero
df_noise = df_noise.rename(
    columns={
        '1-1_small-sounding-engine_presence': 'smallSoundingEngine',
        '1-2_medium-sounding-engine_presence': 'mediumSoundingEngine'
    }
)

In [6]:
df_noise.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62022 entries, 0 to 62021
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   borough               62022 non-null  int64  
 1   block                 62022 non-null  int64  
 2   latitude              62022 non-null  float64
 3   longitude             62022 non-null  float64
 4   year                  62022 non-null  int64  
 5   week                  62022 non-null  int64  
 6   day                   62022 non-null  int64  
 7   hour                  62022 non-null  int64  
 8   smallSoundingEngine   62022 non-null  int64  
 9   mediumSoundingEngine  62022 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 4.7 MB


df_noise contiene las variables que usaremos para el dashborad

Configuracion de credenciales para la conexion a Google Cloud Platform

El archivo 'sturdy-gate-417001-19ab59ab9df1.json' que tiene las credenciales de la cuenta de servicio en GCP se descarga cuando se crea una nueva cuenta de servicio y se agrega una nueva clave.

In [5]:
key_path = '../../sturdy-gate-417001-19ab59ab9df1.json'
client = bigquery.Client.from_service_account_json(key_path)

In [7]:
# id de la tabla para los ruidos de NYC
table_id = 'sturdy-gate-417001.data_clean.noise'

# creamos la estructura de la tabla
schema = []
table = bigquery.Table(table_id, schema=schema)

# enviando la instruccion a bigquery
table = client.create_table(table)

In [15]:
job_config = bigquery.LoadJobConfig(
    # especificamos el esquema de la tabla noise
    schema=[
        # Aqui especificamos el tipo de columna cuyo tipo no se puede detectar automaticamente
        bigquery.SchemaField('borough', 'INTEGER'),
        bigquery.SchemaField('block', 'INTEGER'),
        bigquery.SchemaField('latitude', 'FLOAT'),
        bigquery.SchemaField('longitude', 'FLOAT'),
        bigquery.SchemaField('year', 'INTEGER'),
        bigquery.SchemaField('week', 'INTEGER'),
        bigquery.SchemaField('day', 'INTEGER'),
        bigquery.SchemaField('hour', 'INTEGER'),
        bigquery.SchemaField('1-1_small-sounding-engine_presence', 'INTEGER'),
        bigquery.SchemaField('1-2_medium-sounding-engine_presence', 'INTEGER')
    ],
    write_disposition='WRITE_TRUNCATE' # WRITE_APPEND para sobrescribir
)

In [16]:
# Proceso de ingesta
job = client.load_table_from_dataframe(
    df_noise, #dataframe a cargar en GCP
    table_id, #tabla creada previamente en bigquery para recibir la carga
    job_config=job_config #configuracion de carga
)

In [17]:
job.result()

LoadJob<project=sturdy-gate-417001, location=southamerica-east1, id=ca156e5b-d52d-4b6c-bc95-b84c9ae5cef5>